# 주가지수만들기
--------------
<STEP 1>
   1. TCI 지수 선정 : 일일 확진자(과거데이터 가중치{EMA,MA등 사용}, 거리두기 단계(1~4단계), Additional Wieghted_Value : 누적사망자/누적확진자)
   > (누적사망자/누적확진자) 가 클수록 코로나의 위험도가 증가한다고 판단하여 가중치 값으로 선정
   2. TCI 지수와 코스피, 코스닥 움직임 상관관계 예측
---------------
<STEP 2>
   1. TCI 지수 산출을 위한 데이터 만들기 ('2020.03.10' ~ '2022.02.28')
   > 지수 산출방법 : ```(추가로더적을 부분)``` 4번까지 한다음에 다시 피드백해서 정하기
   2. 코스피, 코스닥 동기간 종가 불러오기
   3. 임의로 만든 TCI와 KOSPI & KOSDAQ 상관관계 비교
   4. 산업군별로 데이터분석
      > 상관성 높은 산업군에서 예측모델 사용권장
      ``예측은 아마도 헬스케어는 + , 운송업은 -``

In [2]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Assign Corona.csv to df
df = pd.read_csv('./datasets/Corona.csv')
dist = pd.read_excel('./datasets/Dist.xlsx')

# Preprocessing Corona Data

In [4]:
# Preprocessing Corona Data
df.columns = ["1", 'Date', 'DeadCnt', 'ConfirmedCnt', "2", "3", "4", "5", "6", "7"]
dist.columns = ['Date', 'Level']
df.drop([str(i) for i in range(1,8)],axis=1,inplace=True) # Drop Unecessary Columns
df = df.sort_values('Date') # Sorting DATE
df['DailyDeadCnt'] = (df['DeadCnt']-df['DeadCnt'].shift(1)).fillna(0) # Create DailyDeadCnt by using difference & fill NAN by 0
df['DailyConfirmedCnt'] = (df['ConfirmedCnt']-df['ConfirmedCnt'].shift(1)).fillna(0) # Create DailyConfirmedCnt by using difference & fill NAN by 0
df['D/C(%)'] = (df['DeadCnt']/df['ConfirmedCnt'])*100 # Create D/C
df.drop(['DeadCnt','ConfirmedCnt'],axis=1,inplace=True) # Drop Unecessary Columns
df.reset_index(drop=True, inplace=True) # Reset index
df['Date'] = df['Date'].str[0:4] + "-" + df['Date'].str[5:7] + "-" +df['Date'].str[8:10] # Change Date style YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True) # Change datatype from object to datetime
df = pd.merge(df, dist, left_on='Date', right_on='Date', how = 'left') # Merge df & Corona Level
df.fillna(3, inplace=True) # ALL NAN index in Level column fill 3


# Preprocessing stock index

In [5]:
# Read Stock Index 
Kospi = fdr.DataReader('KS11') 
Kodaq = fdr.DataReader('KQ11')

In [6]:
# Preprocessing
df_c = df
sp_str = ['Kospi', 'Kosdq']
sp_df = [Kospi, Kodaq]

for i, col in enumerate(sp_df): # Merge Kospi & Kodaq into df_c
    corona_df = pd.merge(df_c, col['Close'], left_on='Date', right_on=col.index, how='inner')
    corona_df.rename(columns={'Close': sp_str[i]}, inplace=True)
    df_c = corona_df


In [7]:
# Check corona_df to finish preprocessing 
corona_df

,Date,DailyDeadCnt,DailyConfirmedCnt,D/C(%),Level,Kospi,Kosdq
0,2020-03-10,0.0,0.0,0.718754,3.0,1962.93,619.97
1,2020-03-11,6.0,242.0,0.773694,3.0,1908.27,595.61
2,2020-03-12,6.0,114.0,0.838734,3.0,1834.33,563.49
3,2020-03-13,1.0,110.0,0.839704,3.0,1771.44,524.00
4,2020-03-16,0.0,74.0,0.910636,3.0,1714.86,504.51
...,...,...,...,...,...,...,...
481,2022-02-22,58.0,99569.0,0.347958,2.0,2706.79,868.11
482,2022-02-23,99.0,171442.0,0.326597,2.0,2719.53,877.33
483,2022-02-24,82.0,170015.0,0.307660,2.0,2648.80,848.21
484,2022-02-25,94.0,165887.0,0.292037,2.0,2676.76,872.98


In [8]:
corona_df['Level'].value_counts()

2.0    256
1.0     93
4.0     74
3.0     63
Name: Level, dtype: int64

# EWM

In [9]:
df_reversed = corona_df.sort_index(ascending=False)
df_reversed = df_reversed.assign(DCC_ewm=df_reversed['DailyConfirmedCnt'].ewm(span=5).mean())
corona_df = corona_df.assign(DCC_ewm=df_reversed['DCC_ewm'])
cd_p = corona_df


In [10]:
cd_p['DCC_ewm'] = np.log1p(cd_p['DCC_ewm'])
cd_p['DailyConfirmedCnt'] = np.log1p(cd_p['DailyConfirmedCnt'])

In [11]:
cd_p

,Date,DailyDeadCnt,DailyConfirmedCnt,D/C(%),Level,Kospi,Kosdq,DCC_ewm
0,2020-03-10,0.0,0.000000,0.718754,3.0,1962.93,619.97,4.605328
1,2020-03-11,6.0,5.493061,0.773694,3.0,1908.27,595.61,5.007455
2,2020-03-12,6.0,4.744932,0.838734,3.0,1834.33,563.49,4.632645
3,2020-03-13,1.0,4.709530,0.839704,3.0,1771.44,524.00,4.571389
4,2020-03-16,0.0,4.317488,0.910636,3.0,1714.86,504.51,4.494434
...,...,...,...,...,...,...,...,...
481,2022-02-22,58.0,11.508616,0.347958,2.0,2706.79,868.11,11.853418
482,2022-02-23,99.0,12.052006,0.326597,2.0,2719.53,877.33,12.020384
483,2022-02-24,82.0,12.043648,0.307660,2.0,2648.80,848.21,11.997293
484,2022-02-25,94.0,12.019068,0.292037,2.0,2676.76,872.98,11.953652


In [12]:
corona_df

,Date,DailyDeadCnt,DailyConfirmedCnt,D/C(%),Level,Kospi,Kosdq,DCC_ewm
0,2020-03-10,0.0,0.000000,0.718754,3.0,1962.93,619.97,4.605328
1,2020-03-11,6.0,5.493061,0.773694,3.0,1908.27,595.61,5.007455
2,2020-03-12,6.0,4.744932,0.838734,3.0,1834.33,563.49,4.632645
3,2020-03-13,1.0,4.709530,0.839704,3.0,1771.44,524.00,4.571389
4,2020-03-16,0.0,4.317488,0.910636,3.0,1714.86,504.51,4.494434
...,...,...,...,...,...,...,...,...
481,2022-02-22,58.0,11.508616,0.347958,2.0,2706.79,868.11,11.853418
482,2022-02-23,99.0,12.052006,0.326597,2.0,2719.53,877.33,12.020384
483,2022-02-24,82.0,12.043648,0.307660,2.0,2648.80,848.21,11.997293
484,2022-02-25,94.0,12.019068,0.292037,2.0,2676.76,872.98,11.953652


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import reg_model
import score_model
from imp import reload
reload(reg_model)
reload(score_model)
from sklearn.model_selection import GridSearchCV

corona_feature1 = cd_p[['DailyDeadCnt', 'DailyConfirmedCnt', 'D/C(%)', 'Level']]
corona_feature2 = cd_p[['DailyDeadCnt', 'DCC_ewm', 'D/C(%)', 'Level']]
corona_target = cd_p['Kospi']

X_train, X_test, y_train, y_test = train_test_split(corona_feature2, corona_target, test_size=0.3, random_state=0)

# # 회귀 모델과 스코어링 모델 선언
# rg = reg_model.RegALL()
# sm = score_model.ScoreALL()

# pred_linr=rg.Linreg(X_train, X_test, y_train)
# pred_logr=rg.Logreg(X_train, X_test, y_train)
# pred_rid=rg.Rid(X_train, X_test, y_train)
# pred_las=rg.Las(X_train, X_test, y_train)

# # 스코어링을 위한 리스트 만들기
# pred_list_name = ['pred_linr', 'pred_logr', 'pred_rid', 'pred_las']
# pred_list_df = [pred_linr, pred_logr, pred_rid, pred_las]

# # 선형회귀에 적용되는 스코어링 적용해보기
# for i, target in enumerate(pred_list_df):
#     print("\n",pred_list_name[i])
#     sm.r2_(y_test,target)
#     sm.evs_(y_test,target)


# model = LinearRegression(normalize=True)


# model.fit(corona_feature1, corona_target)
# print(model.score(corona_feature1, corona_target))

# model.fit(corona_feature2, corona_target)
# print(model.score(corona_feature2, corona_target))

In [17]:
rg = reg_model.RegALL()

rg.Super(LinearRegression, X_train, X_test, y_train)


In [24]:
s[1]

0.5716081449668637